In [1]:
!pip install nltk
!pip install networkx

     |████████████████████████████████| 1.4 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 284 kB 595 kB/s eta 0:00:01
     |████████████████████████████████| 73 kB 1.4 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434675 sha256=84cb3c53aa475f109e43230d0795e0229d8ee500bd833613d33a35df78e4c818
  Stored in directory: /Users/jordansamek/Library/Caches/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
     |████████████████████████████████| 1.6 MB 3.6 MB/s eta 0:00:01


#### Code flow

Input article --> split into sentences --> remove stop words --> build similarity matrix --> generate rank based on matrix --> pick top N sentences for summary

In [2]:
import numpy as np
import networkx as nx
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
print('Setup Complete.')

Setup Complete.


In [43]:
# Generate clean sentences

def read_article(file_name, top_n):
    
    file = open(file_name, "r")
    filedata = file.readlines()
#     article = filedata[0].split(". ")
    sentences = []
    
    for i in range(top_n):
        article = filedata[i].split(". ")
        
    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()
    return sentences

In [44]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
        
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    
    all_words = list(set(sent1 + sent2))
    
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
        
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
        
    return 1 - cosine_distance(vector1, vector2)

In [45]:
# Build Similarity Matrix

def build_similarity_matrix(sentences, stop_words):
    # create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: # ignore if both are same sentences since the cosine will be zero
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
            
    return similarity_matrix

In [46]:
# Generate summary method. This method will call all other 
# helper functions to keep our summarization pipeline going

def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    
    # Step 1 - Read text and tokenize
    sentences = read_article(file_name, top_n)
    
    # Step 2 - Generate similarity matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
    print(sentence_similarity_matrix)
    
    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are", ranked_sentence)
    
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
        
    # Step 5 - Output the summarized text
    print("Summarized Text:\n", ". ".join(summarize_text))

In [47]:
generate_summary(file_name='./data/msft.txt', top_n=2)



[]
Indexes of top ranked_sentence order are []


IndexError: list index out of range